In [1]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

from sklearn.neighbors import LocalOutlierFactor

from DataMacpy import *

DataMacpy Setup


In [2]:
# Veriyi içeri aktaralım.

# veri yolumuzu verelim
file_path = 'train2.csv'

# verimizi okuyoruz. 
df = pd.read_csv(file_path)

print("Setup Complete")

Setup Complete


In [3]:
df.head()

,index,Cinsiyet,Yaş Grubu,Medeni Durum,Eğitim Düzeyi,İstihdam Durumu,Yıllık Ortalama Gelir,Yaşadığı Şehir,En Çok İlgilendiği Ürün Grubu,Yıllık Ortalama Satın Alım Miktarı,...,Eğitime Devam Etme Durumu,Öbek İsmi,Yıllık Ortalama Sepete Atılan Ürün Adedi,Yaş Durumu,Eğitim Durumu,Yaşadığı Şehir Durumu,Yıllık Ortalama Gelir Durumu,Yıllık Ortalama Satın Alım Durumu,Yıllık Ortalama Sipariş Verilen Ürün Adedi Durumu,Yıllık Ortalama Sepete Alınan Ürün Adedi Durumu
0,0,Kadın,31-40,Bekar,Yüksek Lisans Mezunu,Düzenli ve Ücretli Bir İşi Var,748266.440879,Büyük Şehir,Elektronik ve Teknolojik Ürünler,32010.900881,...,Etmiyor,obek_4,102.936904,1,3,2,4,3,3,4
1,1,Erkek,>60,Evli,Lise Mezunu,Kendi İşinin Sahibi,246298.609605,Küçük Şehir,Spor Malzemeleri,4145.729891,...,Etmiyor,obek_5,82.176053,3,2,2,2,1,2,3
2,2,Erkek,18-30,Bekar,Lise Mezunu,Düzenli ve Ücretli Bir İşi Var,268582.732360,Küçük Şehir,Giyim,9924.379414,...,Ediyor,obek_8,141.661426,1,2,2,2,2,3,4
3,3,Erkek,51-60,Evli,Lise Mezunu,Kendi İşinin Sahibi,327721.837865,Kırsal,Giyim,6417.780714,...,Etmiyor,obek_3,17.097397,2,2,1,3,1,2,1
4,4,Erkek,31-40,Evli,Ortaokul Mezunu,İşsiz veya Düzenli Bir İşi Yok,397431.626948,Büyük Şehir,Giyim,7886.663061,...,Etmiyor,obek_3,20.470459,1,1,2,3,1,2,1


In [4]:
# target verilerimiz string olduğu için LabelEncoder yapıyoruz. Her sınıfı bir sayıya atıyoruz.
l_fit = LabelEncoder().fit(df["Öbek İsmi"])
y = l_fit.transform(df["Öbek İsmi"])
#y = pd.DataFrame(y, columns = ["Type_1"]) # dataframe'e çevirmek için bu satırı kullanabiliriz.
y[0:20]

array([3, 4, 7, 2, 2, 7, 2, 4, 2, 5, 4, 0, 6, 2, 6, 4, 6, 5, 5, 5])

In [5]:
# labellerimizi buluyoruz.
y_label = l_fit.inverse_transform([i for i in range(max(y)+1)])
label_and_num = list(zip(y_label, [i for i in range(max(y)+1)]))
label_and_num

[('obek_1', 0),
 ('obek_2', 1),
 ('obek_3', 2),
 ('obek_4', 3),
 ('obek_5', 4),
 ('obek_6', 5),
 ('obek_7', 6),
 ('obek_8', 7)]

In [7]:
df["Öbek İsmi"] = y
df

,index,Cinsiyet,Yaş Grubu,Medeni Durum,Eğitim Düzeyi,İstihdam Durumu,Yıllık Ortalama Gelir,Yaşadığı Şehir,En Çok İlgilendiği Ürün Grubu,Yıllık Ortalama Satın Alım Miktarı,...,Eğitime Devam Etme Durumu,Öbek İsmi,Yıllık Ortalama Sepete Atılan Ürün Adedi,Yaş Durumu,Eğitim Durumu,Yaşadığı Şehir Durumu,Yıllık Ortalama Gelir Durumu,Yıllık Ortalama Satın Alım Durumu,Yıllık Ortalama Sipariş Verilen Ürün Adedi Durumu,Yıllık Ortalama Sepete Alınan Ürün Adedi Durumu
0,0,Kadın,31-40,Bekar,Yüksek Lisans Mezunu,Düzenli ve Ücretli Bir İşi Var,748266.440879,Büyük Şehir,Elektronik ve Teknolojik Ürünler,32010.900881,...,Etmiyor,3,102.936904,1,3,2,4,3,3,4
1,1,Erkek,>60,Evli,Lise Mezunu,Kendi İşinin Sahibi,246298.609605,Küçük Şehir,Spor Malzemeleri,4145.729891,...,Etmiyor,4,82.176053,3,2,2,2,1,2,3
2,2,Erkek,18-30,Bekar,Lise Mezunu,Düzenli ve Ücretli Bir İşi Var,268582.732360,Küçük Şehir,Giyim,9924.379414,...,Ediyor,7,141.661426,1,2,2,2,2,3,4
3,3,Erkek,51-60,Evli,Lise Mezunu,Kendi İşinin Sahibi,327721.837865,Kırsal,Giyim,6417.780714,...,Etmiyor,2,17.097397,2,2,1,3,1,2,1
4,4,Erkek,31-40,Evli,Ortaokul Mezunu,İşsiz veya Düzenli Bir İşi Yok,397431.626948,Büyük Şehir,Giyim,7886.663061,...,Etmiyor,2,20.470459,1,1,2,3,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5455,5455,Kadın,>60,Bekar,Üniversite Mezunu,Emekli,478054.247819,Köy veya Kasaba,Kozmetik,43182.299801,...,Etmiyor,5,51.600436,3,3,1,3,3,2,2
5456,5456,Erkek,51-60,Evli,Doktora Ötesi,Kendi İşinin Sahibi,481852.853651,Köy veya Kasaba,Ev ve Mobilya,42107.996193,...,Etmiyor,5,49.190536,2,4,1,3,3,2,2
5457,5457,Kadın,51-60,Evli,İlkokul Mezunu,Düzenli ve Ücretli Bir İşi Var,309443.687351,Küçük Şehir,Ev ve Mobilya,4655.392445,...,Etmiyor,0,9.537384,2,1,2,3,1,2,1
5458,5458,Kadın,18-30,Bekar,Yüksekokul Mezunu,Düzenli ve Ücretli Bir İşi Var,266604.268016,Büyük Şehir,Ev ve Mobilya,9016.165906,...,Ediyor,7,143.281718,1,2,2,2,2,3,4


In [8]:
object_cols, d = calc_object_cols(df, 0)
d

{'Cinsiyet': 2,
 'Yaş Grubu': 5,
 'Medeni Durum': 2,
 'Eğitim Düzeyi': 9,
 'İstihdam Durumu': 4,
 'Yaşadığı Şehir': 4,
 'En Çok İlgilendiği Ürün Grubu': 5,
 'Eğitime Devam Etme Durumu': 2}

In [9]:
# Verilerimize OrdinalEncoder uyguluyoruz. OneHot denemesi yapılacak!
ordinal_encoder = OrdinalEncoder()
df[object_cols] = ordinal_encoder.fit_transform(df[object_cols])
df

,index,Cinsiyet,Yaş Grubu,Medeni Durum,Eğitim Düzeyi,İstihdam Durumu,Yıllık Ortalama Gelir,Yaşadığı Şehir,En Çok İlgilendiği Ürün Grubu,Yıllık Ortalama Satın Alım Miktarı,...,Eğitime Devam Etme Durumu,Öbek İsmi,Yıllık Ortalama Sepete Atılan Ürün Adedi,Yaş Durumu,Eğitim Durumu,Yaşadığı Şehir Durumu,Yıllık Ortalama Gelir Durumu,Yıllık Ortalama Satın Alım Durumu,Yıllık Ortalama Sipariş Verilen Ürün Adedi Durumu,Yıllık Ortalama Sepete Alınan Ürün Adedi Durumu
0,0,1.0,1.0,0.0,5.0,0.0,748266.440879,0.0,0.0,32010.900881,...,1.0,3,102.936904,1,3,2,4,3,3,4
1,1,0.0,4.0,1.0,3.0,2.0,246298.609605,2.0,4.0,4145.729891,...,1.0,4,82.176053,3,2,2,2,1,2,3
2,2,0.0,0.0,0.0,3.0,0.0,268582.732360,2.0,2.0,9924.379414,...,0.0,7,141.661426,1,2,2,2,2,3,4
3,3,0.0,3.0,1.0,3.0,2.0,327721.837865,3.0,2.0,6417.780714,...,1.0,2,17.097397,2,2,1,3,1,2,1
4,4,0.0,1.0,1.0,4.0,3.0,397431.626948,0.0,2.0,7886.663061,...,1.0,2,20.470459,1,1,2,3,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5455,5455,1.0,4.0,0.0,7.0,1.0,478054.247819,1.0,3.0,43182.299801,...,1.0,5,51.600436,3,3,1,3,3,2,2
5456,5456,0.0,3.0,1.0,1.0,2.0,481852.853651,1.0,1.0,42107.996193,...,1.0,5,49.190536,2,4,1,3,3,2,2
5457,5457,1.0,3.0,1.0,8.0,0.0,309443.687351,2.0,1.0,4655.392445,...,1.0,0,9.537384,2,1,2,3,1,2,1
5458,5458,1.0,0.0,0.0,6.0,0.0,266604.268016,0.0,1.0,9016.165906,...,0.0,7,143.281718,1,2,2,2,2,3,4


In [10]:
lof = LocalOutlierFactor()
lof.fit(df)
pred = lof.fit_predict(df)
pred

array([1, 1, 1, ..., 1, 1, 1])

In [11]:
np.unique(pred, return_counts=True)

(array([-1,  1]), array([  68, 5392], dtype=int64))

In [16]:
df["inlier"] = pred

In [17]:
rslt_df = df[df['inlier'] == -1]
rslt_df

,index,Cinsiyet,Yaş Grubu,Medeni Durum,Eğitim Düzeyi,İstihdam Durumu,Yıllık Ortalama Gelir,Yaşadığı Şehir,En Çok İlgilendiği Ürün Grubu,Yıllık Ortalama Satın Alım Miktarı,...,Öbek İsmi,Yıllık Ortalama Sepete Atılan Ürün Adedi,Yaş Durumu,Eğitim Durumu,Yaşadığı Şehir Durumu,Yıllık Ortalama Gelir Durumu,Yıllık Ortalama Satın Alım Durumu,Yıllık Ortalama Sipariş Verilen Ürün Adedi Durumu,Yıllık Ortalama Sepete Alınan Ürün Adedi Durumu,inlier
155,155,1.0,0.0,0.0,4.0,3.0,189348.802058,0.0,0.0,9025.100309,...,7,154.036246,1,1,2,2,2,3,4,-1
184,184,0.0,3.0,1.0,7.0,2.0,494757.358828,3.0,4.0,47177.288486,...,5,54.719631,2,3,1,3,3,2,2,-1
204,204,1.0,4.0,1.0,5.0,1.0,487446.768615,1.0,1.0,33409.864716,...,5,61.779529,3,3,1,3,3,2,3,-1
311,311,0.0,0.0,0.0,6.0,3.0,166403.198497,2.0,3.0,8813.629018,...,7,111.491624,1,2,2,2,2,3,4,-1
358,358,1.0,0.0,1.0,6.0,0.0,188104.232755,2.0,2.0,8850.791555,...,7,108.224484,1,2,2,2,2,3,4,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999,4999,1.0,0.0,1.0,3.0,0.0,43922.988693,1.0,3.0,4169.949102,...,0,10.408325,1,2,1,1,1,1,1,-1
5100,5100,0.0,0.0,1.0,3.0,0.0,45695.091498,2.0,4.0,6740.639692,...,2,16.284374,1,2,2,1,1,2,1,-1
5130,5130,0.0,3.0,0.0,1.0,0.0,299549.793156,0.0,0.0,27327.309076,...,6,103.867181,2,4,2,2,3,2,4,-1
5215,5215,0.0,0.0,0.0,1.0,3.0,548845.696664,0.0,4.0,31824.835159,...,3,98.404522,1,4,2,4,3,3,3,-1


In [18]:
rslt_df.index

Int64Index([ 155,  184,  204,  311,  358,  383,  439,  456,  542,  565,  617,
             634,  849,  886,  927, 1112, 1145, 1172, 1193, 1252, 1431, 1479,
            1538, 1674, 1716, 1857, 1892, 2009, 2072, 2099, 2257, 2299, 2315,
            2413, 2419, 2422, 2519, 2602, 2717, 2832, 2910, 3083, 3136, 3140,
            3300, 3394, 3419, 3520, 3791, 3940, 3994, 4041, 4102, 4184, 4204,
            4227, 4230, 4275, 4395, 4564, 4624, 4728, 4775, 4999, 5100, 5130,
            5215, 5277],
           dtype='int64')

In [19]:
df = df.drop(index=rslt_df.index)

In [20]:
y = df["Öbek İsmi"]
y

0       3
1       4
2       7
3       2
4       2
       ..
5455    5
5456    5
5457    0
5458    7
5459    7
Name: Öbek İsmi, Length: 5392, dtype: int32

In [21]:
# özniteliklerimizi belirleyelim.
X = df
X.drop(["Öbek İsmi"], axis=1, inplace=True) # tüm verilerimizden hedef kolonumuzu çıkarıyoruz.
# Number kolonumuzu da gereksiz olduğu eğitimde gereksiz olduğu için çıkarıyoruz.
X

,index,Cinsiyet,Yaş Grubu,Medeni Durum,Eğitim Düzeyi,İstihdam Durumu,Yıllık Ortalama Gelir,Yaşadığı Şehir,En Çok İlgilendiği Ürün Grubu,Yıllık Ortalama Satın Alım Miktarı,...,Eğitime Devam Etme Durumu,Yıllık Ortalama Sepete Atılan Ürün Adedi,Yaş Durumu,Eğitim Durumu,Yaşadığı Şehir Durumu,Yıllık Ortalama Gelir Durumu,Yıllık Ortalama Satın Alım Durumu,Yıllık Ortalama Sipariş Verilen Ürün Adedi Durumu,Yıllık Ortalama Sepete Alınan Ürün Adedi Durumu,inlier
0,0,1.0,1.0,0.0,5.0,0.0,748266.440879,0.0,0.0,32010.900881,...,1.0,102.936904,1,3,2,4,3,3,4,1
1,1,0.0,4.0,1.0,3.0,2.0,246298.609605,2.0,4.0,4145.729891,...,1.0,82.176053,3,2,2,2,1,2,3,1
2,2,0.0,0.0,0.0,3.0,0.0,268582.732360,2.0,2.0,9924.379414,...,0.0,141.661426,1,2,2,2,2,3,4,1
3,3,0.0,3.0,1.0,3.0,2.0,327721.837865,3.0,2.0,6417.780714,...,1.0,17.097397,2,2,1,3,1,2,1,1
4,4,0.0,1.0,1.0,4.0,3.0,397431.626948,0.0,2.0,7886.663061,...,1.0,20.470459,1,1,2,3,1,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5455,5455,1.0,4.0,0.0,7.0,1.0,478054.247819,1.0,3.0,43182.299801,...,1.0,51.600436,3,3,1,3,3,2,2,1
5456,5456,0.0,3.0,1.0,1.0,2.0,481852.853651,1.0,1.0,42107.996193,...,1.0,49.190536,2,4,1,3,3,2,2,1
5457,5457,1.0,3.0,1.0,8.0,0.0,309443.687351,2.0,1.0,4655.392445,...,1.0,9.537384,2,1,2,3,1,2,1,1
5458,5458,1.0,0.0,0.0,6.0,0.0,266604.268016,0.0,1.0,9016.165906,...,0.0,143.281718,1,2,2,2,2,3,4,1


In [22]:
# Verilerimizi train ve validation olarak ayırıyoruz.
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [23]:
from sklearn.neighbors import KNeighborsClassifier

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble import ExtraTreesClassifier

from sklearn.ensemble import GradientBoostingClassifier

from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier, VotingClassifier

import lightgbm as lgb

from lightgbm import LGBMClassifier

from catboost import CatBoostClassifier

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [24]:
model_list = [
    
    LogisticRegression(multi_class='multinomial', random_state=1, solver="newton-cg"), # 0
    
    KNeighborsClassifier(n_neighbors=3), # 1
    
    DecisionTreeClassifier(), # 2
    
    RandomForestClassifier(), # 3
    
    ExtraTreesClassifier(), # 4
    
    HistGradientBoostingClassifier(), # 5
    
    VotingClassifier(estimators=[('catb', CatBoostClassifier(iterations=3500, task_type="GPU", verbose=0)),
                                 ('rf', RandomForestClassifier()),
                                 ('lgb', LGBMClassifier())],
                                 voting='soft'), # 6
    
    LGBMClassifier(), # 7
    
    CatBoostClassifier(iterations=3500, task_type="GPU", verbose=0) # 8
    
]

fast_model_select(model_list, X, y)

Model 0 Accuracy: %83.23663486439413
Model 1 Accuracy: %79.8964986962781
Model 2 Accuracy: %74.68602658119482
Model 3 Accuracy: %95.43775736308339
Model 4 Accuracy: %95.4191976121369
Model 5 Accuracy: %93.41603409794557
Model 6 Accuracy: %94.7699983637707
Model 7 Accuracy: %92.35894331363049
Model 8 Accuracy: %95.08533874588906
